## Scrape a URL with captcha

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

__Set URL to scrape__

In [2]:
#url = f"https://www.amazon.es/Cecotec-Aspirador-Tecnolog%C3%ADa-girosc%C3%B3pica-multifunci%C3%B3n/dp/B0CGRHLHJZ/ref=sr_1_6?keywords=conga&qid=1701850129&sr=8-6&ufe=app_do%3Aamzn1.fos.7911a93e-3532-4810-a632-e84fe123af2b"
url = 'https://www.amazon.com/s?k=Logitech+G203'

__Run in Selenium Grid__

In [6]:
options = Options()
#options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
chrome = webdriver.Remote(
    command_executor='http://localhost:4444/wd/hub',
    options=options,
)
chrome.get(url)

In [9]:
chrome.refresh()

__Run in standalone__

In [3]:
chrome = webdriver.Chrome()
chrome.get(url)
captcha_url = chrome.find_elements(By.TAG_NAME, 'img')[0].get_attribute('src')
print(captcha_url)

https://fls-na.amazon.com/1/batch/1/OP/ATVPDKIKX0DER:140-1222902-2170639:NYXE8BSF4G6C0572P993$uedata=s:%2Frd%2Fuedata%3Fstaticb%26id%3DNYXE8BSF4G6C0572P993:0


__Take a screenshot__

In [12]:
options = Options()
options.add_argument('--headless')
chrome = webdriver.Chrome(options=options)
chrome.set_window_size('1920', '1080')
chrome.get(url)
chrome.save_screenshot('screenshot.png')

True

## TOR

In [1]:
from tbselenium.tbdriver import TorBrowserDriver
with TorBrowserDriver("/path/to/tor-browser/") as driver:
    driver.get(url)


TBDriverPathError: TBB path is not a directory /path/to/tor-browser/

__Show captcha image and mock it if you need it__

In [6]:
import requests
from IPython.display import Image, display
respuesta = requests.get(captcha_url)
display(Image(respuesta.content))

ModuleNotFoundError: No module named 'models'

## Try to solve with amazoncaptcha package

In [7]:
from amazoncaptcha import AmazonCaptcha

captcha = AmazonCaptcha.fromlink(captcha_url)
captcha = captcha.solve()
print(captcha)

ModuleNotFoundError: No module named 'models'

## Write captcha and send

In [4]:
chrome.find_element(By.NAME, 'field-keywords').send_keys(captcha)
chrome.find_element(By.TAG_NAME, 'button').click()

NameError: name 'captcha' is not defined

__accept cookies__

In [ ]:
cookies_dialog = chrome.find_element(By.ID, 'sp-cc')
cookies_dialog.find_element(By.ID, 'sp-cc-accept').click()

## Scrape Reviews

__Read statistics__

In [ ]:
reviews_section = chrome.find_element(By.ID, 'reviewsMedley')
rating_histogram = reviews_section.find_element(By.ID, 'cm_cr_dp_d_rating_histogram')
total_reviews = rating_histogram.find_element(By.CLASS_NAME, 'averageStarRatingNumerical').text
print(total_reviews)

average_reviews_div = rating_histogram.find_element(By.CLASS_NAME, 'AverageCustomerReviews')
spans = average_reviews_div.find_elements(By.TAG_NAME, 'span')
for span in spans:
    print(span.text)


histogram_table = reviews_section.find_element(By.ID, 'histogramTable')
histogram_rows = histogram_table.find_elements(By.TAG_NAME, 'tr')

five_stars = histogram_rows[0].find_elements(By.TAG_NAME, 'td')[2].text
print(f"5 stars :{five_stars}")

four_stars = histogram_rows[1].find_elements(By.TAG_NAME, 'td')[2].text
print(f"4 stars :{four_stars}")

three_stars = histogram_rows[2].find_elements(By.TAG_NAME, 'td')[2].text
print(f"3 stars :{three_stars}")

two_stars = histogram_rows[3].find_elements(By.TAG_NAME, 'td')[2].text
print(f"2 stars :{two_stars}")

one_stars = histogram_rows[4].find_elements(By.TAG_NAME, 'td')[2].text
print(f"1 stars :{one_stars}")


__Read features__


In [36]:
try:
    product_overview_div = chrome.find_element(By.ID, 'poExpander')
    
    try:
        product_overview_table = product_overview_div.find_elements(By.TAG_NAME, 'table')[0]
        rows = product_overview_table.find_elements(By.TAG_NAME, 'tr')
        product_overview = {}
        for row in rows:
            columns = row.find_elements(By.TAG_NAME, 'td')
            product_overview[columns[0].text] = columns[1].text
        print(product_overview)
    except:
        print("No table find by tag name 'table' in product overview div")

except:
    print("No product overview find by id 'poExpander'")

try:
    features = chrome.find_element(By.ID, 'featurebullets_feature_div').text
    print(f"Features: {features}")
except:
    print("No features div find by id 'featurebullets_feature_div'")

try:
    product_specs_table = chrome.find_element(By.ID, 'productDetails_techSpec_section_1')
    product_specs = {}
    rows = product_specs_table.find_elements(By.TAG_NAME, 'tr')
    for row in rows:
        try:
            field = row.find_element(By.TAG_NAME, 'th')
            value = row.find_element(By.TAG_NAME, 'td')
            product_specs[field.text] = value.text
        except:
            print("No th or td tag find in row")
    print(product_specs)
        
except:
    print("No product specs table find by id 'productDetails_techSpec_section_1'")



try:
    product_description = chrome.find_element(By.ID, 'productDescription').text
    print(f"Product description: {product_description}")
except:
    print("No product description find by id 'productDescription'")


{'Marca': 'Logitech', 'Dispositivos compatibles': 'PC', 'Tecnología de conectividad': 'USB 2.0', 'Descripción del teclado': 'Gaming', 'Usos Recomendados Para Producto': 'Gaming', '': ''}
Features: Sobre este artículo
Lleva tus habilidades de juego al siguiente nivel: el Logitech G413 TKL SE es un teclado sin llave con características de juego y la durabilidad y el rendimiento necesarios para competir
Teclas PBT: resistente al calor y al desgaste, este teclado para juegos de computadora cuenta con el material más duradero utilizado en el diseño de teclas
Interruptores mecánicos táctiles: el rendimiento sin concesiones siempre está al alcance con este teclado para juegos con cable
Color, material y acabado premium: mejora tu configuración de juego con este teclado retroiluminado con una elegante carcasa superior de aluminio cepillado negro e iluminación LED blanca
Rendimiento anti-ghosting de 6 teclas: experimente una entrada de teclas confiable con este teclado anti-ghosting en comparac

__show all reviews__

In [6]:
footer = chrome.find_element(By.ID, 'reviews-medley-footer')
all_reviews_link = footer.find_element(By.TAG_NAME, 'a')
all_reviews_link.click()

__read reviews__

In [14]:
global_info = chrome.find_element(By.ID, 'filter-info-section').text
print(global_info)

reviews = chrome.find_elements(By.CLASS_NAME, 'review')
print(f"There are {len(reviews)} reviews")
for review in reviews:
    review_id = review.get_attribute('id')
    print('//////////////////////////////////')
    print(f"ID: {review_id}")

    user_profile = review.find_element(By.CLASS_NAME, 'a-profile')
    username = user_profile.find_element(By.CLASS_NAME, 'a-profile-name').text
    print(f"username: {username}")
    userAvatar = user_profile.find_element(By.TAG_NAME, 'img').get_attribute('src')
    print(f"userAvatar: {userAvatar}")
    stars_classes = review.find_element(By.CLASS_NAME, 'review-rating').get_attribute('class')
    stars = stars_classes.split('a-star-')[1].split(' ')[0]
    print(f"Rating: {stars}")

    date = review.find_element(By.CLASS_NAME, 'review-date').text
    print(f"Date: {date}")
    
    title_div = review.find_element(By.CLASS_NAME, 'review-title')
    title = title_div.text
    print(f"Title: {title}")
    review_url = title_div.get_attribute('href')
    print(f"Review URL: {review_url}")
    content = review.find_element(By.CLASS_NAME, 'review-text-content').text
    print(f"Content: {content}")
    print('//////////////////////////////////')
    print()

3 calificaciones totales, 1 con opinión
There are 1 reviews
//////////////////////////////////
ID: R320XM916LXMCG
username: Amazon Customer
userAvatar: https://images-na.ssl-images-amazon.com/images/S/amazon-avatars-global/default._CR0,0,1024,1024_SX48_.png
Rating: 5
Date: Calificado en Estados Unidos el 18 de diciembre de 2023
Title: the product is good
Review URL: https://www.amazon.com/-/es/gp/customer-reviews/R320XM916LXMCG/ref=cm_cr_arp_d_rvw_ttl?ie=UTF8&ASIN=B0CB6WKPLN
Content: They have sent the product earlier than expected
//////////////////////////////////



__next page__

In [7]:
chrome.find_element(By.CLASS_NAME, 'a-last').click()

## Scrape products

In [4]:
results = chrome.find_element(By.CLASS_NAME, 's-result-list')
products = results.find_elements(By.CLASS_NAME, 's-result-item')
for product in products:
    uuid = product.get_attribute('data-uuid')
    asin = product.get_attribute('data-asin')
    if uuid:
        print(f"UUID: {uuid}")
        print(f"ASIN: {asin}")
        try:
            sponsored = product.find_element(By.CLASS_NAME, 'puis-sponsored-label-info-icon')
            sponsored = True
        except:
            sponsored = False
        
        print(f"Sponsored: {sponsored}")

        title_h2 = product.find_element(By.TAG_NAME, 'h2')
        title = title_h2.text
        print(f"Title: {title}")

        url = title_h2.find_element(By.TAG_NAME, 'a').get_attribute('href')
        print(f"Url: {url}")

        image = product.find_element(By.TAG_NAME, 'img').get_attribute('src')
        print(f"Image: {image}")
        try:
            price = product.find_element(By.CLASS_NAME, 'a-price').text
            #price = price.find_element(By.CLASS_NAME, 'a-offscreen').text
            print(f"Price: {price}")
        except:
            print("No price")

        try:
            reviews_div = product.find_element(By.CLASS_NAME, 'a-row.a-size-small')
            reviews = reviews_div.find_element(By.CLASS_NAME, 'a-size-base').text
            print(f"Reviews: {reviews}")

            rating_div = reviews_div.find_element(By.TAG_NAME, 'span')
            rating = rating_div.get_attribute('aria-label')
            print(f"Rating: {rating}")
        except:
            print("No reviews")        


UUID: 35be4862-3462-4efc-b407-978fbd0a7e9d
ASIN: B07YN82X3B
Sponsored: False
Title: Logitech G203 RGB - Mouse cableado para videojuegos
Url: https://www.amazon.com/-/es/Logitech-G203-RGB-cableado-videojuegos/dp/B07YN82X3B/ref=sr_1_1?keywords=Logitech+G203&qid=1706345498&sr=8-1
Image: https://m.media-amazon.com/images/I/61UxfXTUyvL._AC_UY218_.jpg
Price: US$29
85
Reviews: 16,026
Rating: 4.7 de 5 estrellas
UUID: 0ed343dc-256b-4fd6-b7c8-6ee560e412a3
ASIN: B091V1LJ4Q
Sponsored: False
Title: Logitech G 203 LIGHTSYNC - Ratón para juegos con cable, 8,000 DPI, efecto óptico arcoíris RGB, 6 botones programables, memoria integrada, computadora PC/Mac, compatible con laptop, color negro (renovado)
Url: https://www.amazon.com/-/es/Logitech-203-LIGHTSYNC-programables-computadora/dp/B091V1LJ4Q/ref=sr_1_2?keywords=Logitech+G203&qid=1706345498&sr=8-2
Image: https://m.media-amazon.com/images/I/51MVbr+ZQML._AC_UY218_.jpg
Price: US$21
99
Reviews: 296
Rating: 4.7 de 5 estrellas
UUID: 66b9075a-7d98-43bc-b6b

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"h2"}
  (Session info: chrome=121.0.6167.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF61AED5E42+3538674]
	(No symbol) [0x00007FF61AAF4C02]
	(No symbol) [0x00007FF61A9A5AEB]
	(No symbol) [0x00007FF61A9EBF4E]
	(No symbol) [0x00007FF61A9EC0CC]
	(No symbol) [0x00007FF61A9E12EC]
	(No symbol) [0x00007FF61AA0F0EF]
	(No symbol) [0x00007FF61A9E124A]
	(No symbol) [0x00007FF61AA0F2C0]
	(No symbol) [0x00007FF61AA2BDE3]
	(No symbol) [0x00007FF61AA0EE53]
	(No symbol) [0x00007FF61A9DF514]
	(No symbol) [0x00007FF61A9E0631]
	GetHandleVerifier [0x00007FF61AF06CAD+3738973]
	GetHandleVerifier [0x00007FF61AF5C506+4089270]
	GetHandleVerifier [0x00007FF61AF54823+4057299]
	GetHandleVerifier [0x00007FF61AC25C49+720121]
	(No symbol) [0x00007FF61AB0126F]
	(No sym

In [23]:
links = chrome.find_elements(By.TAG_NAME, 'a')
buttons = links.find_elements(By.CLASS_NAME, 'a-button-text')
for b in buttons:
    print(f"Button -> {b.text}")

## Close session

In [5]:
chrome.quit()